In [1]:
import numpy as np
import pandas as pd 
import pickle
import os
import pandas as pd
import numpy as np
import math
import datetime
import sys
import sklearn
import scipy.stats as stats
from sklearn.pipeline import Pipeline
from sklearn import linear_model, decomposition
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
# Added version check for recent scikit-learn 0.18 checksok ca
from distutils.version import LooseVersion as Version
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from collections import OrderedDict
from sklearn.svm import SVC # "Support Vector Classifier"
from sklearn.decomposition import FastICA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
from math import sqrt
from sklearn.datasets import make_circles
import seaborn as sns
import matplotlib.pyplot as plt
import pylab as pl
from sklearn.model_selection import train_test_split
%matplotlib inline

###ticker###
ticker='VOD' #enter symbol you want - this will be made for various

# paths
_pkl_path =('/home/ak/Documents/Data/features_models/')#pkl path
_graphs_path= os.path.join(_pkl_path, 'graphs')
_models_path = '/home/ak/Documents/features_test/models/'

###predicitions path
_predictions_path ='/home/ak/Documents/features_models/predictions/'

sys.path.append('/home/ak/Documents/Research/hsmm/hsmm/')

sys.path.append('/home/ak/Documents/Research/hsmm/hsmm/')
sys.path.append('/home/ak/Documents/Research/QFPaper/aknotebooks/classification/convenience_functions/')

/home/ak/Envs/DataAnalysis/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ak/Envs/DataAnalysis/local/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
symbols_list= ['RB','CPI','AAL','IOG','CNA','VOD'] #can expand this after

In [3]:
##useful functions##
##re-write some of these###
def off_set(array_1, array_2):
    off_set= array_1.__len__()- array_2.__len__()
    off_set_abs =np.abs(off_set)
    return off_set_abs

def simpleInputs(_X,_y):
    _offset =np.abs(off_set(_X,_y))
    if _offset==0:
        X = _X.as_matrix().astype(np.float)
        y=_y.astype(np.int)
    else:
        X= _X[:-_offset].as_matrix().astype(np.float)
        y = _y.astype(np.int)
    return X, y

def ensure_dir(file_path): #ensure a dictory exists otherwise create it
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        print('New Directory Created',file_path)
    else:
        print('Directory Exists!check')

def symbol_path_(_main_path, _symb): #create directory for each symbol!
    _main_path = _ftse
    _symbol_path = os.path.join(_main_path,_symb+'/')
    print(_symbol_path)
    ensure_dir(_symbol_path)
    return _symbol_path

def symbol_features_path(_main_path, _symb):
    _symb_features_path = os.path.join(_main_path,_symb+'/')
    print(_symb_features_path)
    ensure_dir(symbol_features_path)
    return symbol_features_path




In [4]:

###main code###

##load data
with open(_pkl_path + ticker+'_features.pickle', 'rb') as handle:
    ticker_features_ = pickle.load(handle)
with open(_pkl_path + ticker+'_labelled_data.pickle', 'rb') as handle:
    ticker_labelled_data_ = pickle.load(handle)

###the following bit is a bit redundant###
_data = ticker_labelled_data_  # data
_features = ticker_features_  # features
_keys = _data.keys()  # dates

# ###scalers###
# mms = MinMaxScaler()
# stdsc = StandardScaler()
# _clfs=[]
# _xr=len(_keys)-1
# print("#of models:",_xr)
# print("for ticker:", ticker)
# for _idx in xrange(0, _xr):
#     # various types of labels
#     y_1 = _data[_keys[_idx]].iloc[:, 4].dropna().astype(int).values  # label_control_chart_q_30_q_70_window_5
#     y_2 = _data[_keys[_idx]].iloc[:, 5].dropna().astype(
#         int).values  # label_simple_ep_window_5_thresh_0.05- Last 4 are NaN
#     y_3 = _data[_keys[_idx]].iloc[:, 6].dropna().astype(
#         int).values  # label_simple_ep_window_5_thresh_0.00-last 4 are NaN
#     duration = _data[_keys[_idx]].iloc[:, 3].dropna().astype(int).values  # can be used for regression

#     # feature set
#     _gamma = _features[_keys[_idx]][2]
#     _csi = _features[_keys[_idx]][3]
#     _info = _features[_keys[_idx]][1]
#     _fischer = _features[_keys[_idx]][0]
#     short_rolling = _data[_keys[_idx]]['TradedPrice'].rolling(window=5).mean()
#     long_rolling = _data[_keys[_idx]]['TradedPrice'].rolling(window=15).mean()
#     ma_signal = (long_rolling - short_rolling).fillna(0)
#     df_new = pd.concat([_csi, _gamma, _fischer, short_rolling], axis=1).dropna()

#     X, y = simpleInputs(df_new, y_3)

#     print("number of classes:",len(np.unique(y)))  # of classes

#     X_mms = X = mms.fit_transform(X)
#     X_train, X_test, y_train, y_test = train_test_split(X_mms, y, test_size=.1, random_state=0)

#     #     # # Train a SVM classification model
#     param_grid = dict(kernel=["rbf"],
#                       C=[1,5,10,25,50],
#                       gamma=[0.0001,0.001,0.01])

#     clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, verbose=1,n_jobs=-1,cv=5)

#     clf.fit(X_train, y_train)
#     filename = _models_path + str(ticker) + '_' + str(_keys[_idx]) + '_clf' + '.joblib.pkl'
#     pickle.dump(clf, open(filename, 'wb'))
#     print('SVM Best Params & Score:')
#     print clf.best_params_, clf.best_score_
#     _clfs.append(clf)
#     clf_files = _models_path + str(ticker) + '_clfs_'  + '.pkl'
#     pickle.dump(_clfs, open(clf_files, 'wb'))
#     print("#of models   left:",_xr -_idx
    # X_1_offset= _set_1[:-_offset].as_matrix().astype(np.float)
    # #define train and test labels - redundant
    # X_train_std = stdsc.fit_transform(X_1_offset)
    # y_train =y_1
    #
    #
    # n_components= [2,4,6]
    # parameters = {
    # "estimator__C": [1,2,4,8],
    # "estimator__kernel": ["poly","rbf"],
    # "estimator__degree":[0.001, 0.01, 1, 3, 4],
    # "pca__n_components": n_components}
    #
    # logistic = linear_model.LogisticRegression()
    # pca = decomposition.PCA()
    # pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])
    # pca.fit(X_train_std)
    #
    # n_components = [2,4]
    # Cs = np.logspace(-4, 4, 3)
    # estimator = GridSearchCV(pipe, dict(pca__n_components=n_components,logistic__C=Cs))
    # estimator.fit(X_train_std, y_train)
    # print('Logistic Best Score:')
    # print(estimator.best_score_)
    #
    #
    #
    # pca = decomposition.RandomizedPCA(n_components=3, whiten=True).fit(X_train_std)
    #
    # X_train_pca = pca.transform(X_train_std)
    #
    # # Train a SVM classification model
    # param_grid = dict(kernel =["rbf","linear"],
    #                   C=[1, 5, 10, 50, 100],
    #                   gamma=[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1])
    # # clf = GridSearchCV(SVC(kernel='rbf'), param_grid,
    # #                verbose=1)
    # clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, verbose=1)
    #
    # clf.fit(X_train_pca, y_train)
    # print('SVM Best Params & Score:')
    # print clf.best_params_, clf.best_score_
    #
    # filename = _models_path+str(ticker)+'_'+str(_keys[1])+'_classifier'+'.joblib.pkl'
    # pickle.dump(clf, open(filename, 'wb'))

In [5]:
_keys

['20180302',
 '20180315',
 '20180316',
 '20180306',
 '20180329',
 '20180313',
 '20180312',
 '20180320',
 '20180308',
 '20180327',
 '20180319',
 '20180321',
 '20180322',
 '20180323',
 '20180314',
 '20180301',
 '20180309',
 '20180328',
 '20180305',
 '20180326',
 '20180307']

In [5]:
_keys[1]
#print np.shape(_features[_keys[1]])
print _data[_keys[1]].__len__()
_lengths_=[_features[_keys[1]][1].__len__(), _features[_keys[1]][2].__len__(),_features[_keys[1]][3].__len__()]
min_length_=min(_lengths_)
print min_length_

7744
7743


In [ ]:
_data[_keys[1]].head(10)
_data_columns_list=_data[_keys[1]].head(10).columns.values.tolist()
_data_columns_list[6]

In [29]:
for _idx, _ in enumerate(_keys):
    print _idx, _
    test=_data[_keys[_idx]].iloc[:, 6].dropna().astype(int).values
    print len(test)

0 20180302
10745
1 20180315
7725
2 20180316
9238
3 20180306
9075
4 20180329
9251
5 20180313
12316
6 20180312
6959
7 20180320
13000
8 20180308
9425
9 20180327
10753
10 20180319
11058
11 20180321
13972
12 20180322
22116
13 20180323
21913
14 20180314
10258
15 20180301
10378
16 20180309
10352
17 20180328
14413
18 20180305
9000
19 20180326
11979
20 20180307
9623


In [7]:
mms = MinMaxScaler()
stdsc = StandardScaler()
_clfs = []
_reports=[]
_idx=1
# various types of labels
y_1 = _data[_keys[_idx]].iloc[:, 4].dropna().astype(int).values  # label_control_chart_q_30_q_70_window_5
y_2 = _data[_keys[_idx]].iloc[:, 5].dropna().astype(int).values  # label_simple_ep_window_5_thresh_0.05- Last 4 are NaN
y_3 = _data[_keys[_idx]].iloc[:, 6].dropna().astype(int).values  # label_simple_ep_window_5_thresh_0.00-last 4 are NaN
duration = _data[_keys[_idx]].iloc[:, 3].dropna().astype(int).values  # can be used for regression

# feature set
_gamma = _features[_keys[_idx]][2]
_csi = _features[_keys[_idx]][3]
_info = _features[_keys[_idx]][1]
_fischer = _features[_keys[_idx]][0]


In [13]:
#label
y = y_3
X, y = simpleInputs(_csi,y)
X_mms = X = mms.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_mms, y, test_size=.1, random_state=0)
## Train a SVM classification model
param_grid = dict(kernel=["rbf"], C=[1,5,10,25,50], gamma=[0.0001,0.001,0.01])
clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, verbose=1,n_jobs=-1,cv=5)            
clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   59.2s finished


In [14]:
print('SVM Best Params & Score:')
print clf.best_params_, clf.best_score_

SVM Best Params & Score:
{'kernel': 'rbf', 'C': 25, 'gamma': 0.0001} 0.63247986191


In [21]:
print classification_report(y_test, y_pred)
print accuracy_score(y_test, y_pred)
# _reports.append(classification_report(y_test, y_pred))
# _clfs.append(clf)

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00       497
          1       0.36      1.00      0.53       276

avg / total       0.13      0.36      0.19       773

0.35705045278137126


In [ ]:
simple_model(_data, _features,_keys)

In [ ]:
_idx=1
_data[_keys[_idx]].iloc[:,6].dropna().astype(int).values

In [19]:
def plot_dataset(X, y, axes):
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
    plt.axis(axes)
    plt.grid(True, which='both')
    plt.xlabel(r"$x_1$", fontsize=20)
    plt.ylabel(r"$x_2$", fontsize=20, rotation=0)

In [ ]:
import pickle
class DataFuncs():
    def __init__(self, _pkl_path='/home/ak/Documents/features_test/',_graphs_path='/home/ak/Documents/features_test/graphs/',_models_path='/home/ak/Documents/features_test/models/',_predictions_path='/home/ak/Documents/features_test/predictions/', ticker='C.N'):
        # Where to save the figures
        self._pkl_path = _pkl_path
        self.ticker = ticker  # enter symbol you want - this will be made for variou
        self._graphs_path = _graphs_path
        self._models_path = _models_path
        self._predictions_path = _predictions_path
#     def __init__(self,_pkl_path,_graphs_path,_models_path,_predictions_path, ticker='C.N'):
#         # Where to save the figures
#         self._pkl_path = ('/home/ak/Documents/features_test/')
#         self.ticker = ticker  # enter symbol you want - this will be made for variou
#         self._graphs_path = '/home/ak/Documents/features_test/graphs/'
#         self._models_path = '/home/ak/Documents/features_test/models/'
#         self._predictions_path = '/home/ak/Documents/features_test/predictions/'

    def off_set(self, array_1, array_2):
        off_set= len(array_1)-len(array_2)
        return off_set

    def simpleInputs(self,_X,_y):
        _offset =np.abs(self.off_set(_X,_y))
        if _offset==0:
            X = _X.as_matrix().astype(np.float)
            y=_y.astype(np.int)
        else:
            X= _X[:-_offset].as_matrix().astype(np.float)
            y = _y.astype(np.int)
        return X, y
##load data
    def load_data(self):
        with open(self._pkl_path + self.ticker+'_features.pickle', 'rb') as handle:
            ticker_features_ = pickle.load(handle)
        with open(self._pkl_path + self.ticker+'_labelled_data.pickle', 'rb') as handle:
            ticker_labelled_data_ = pickle.load(handle)

        return ticker_features_, ticker_labelled_data_


In [ ]:
from sklearn.svm import SVC

gamma1, gamma2 = 0.1, 5
C1, C2 = 0.001, 1000
hyperparams = (gamma1, C1), (gamma1, C2), (gamma2, C1), (gamma2, C2)
X= X_train
y= y_train
svm_clfs = []
for gamma, C in hyperparams:
    rbf_kernel_clf=svm.SVC(kernel="rbf", gamma=gamma, C=C)
#     rbf_kernel_svm_clf = Pipeline((
#             ("scaler", StandardScaler()),
#             ("svm_clf", SVC(kernel="rbf", gamma=gamma, C=C))
#         ))
    rbf_kernel_clf.fit(X, y)
    svm_clfs.append(rbf_kernel_clf)

plt.figure(figsize=(11, 7))
plt.show()

In [ ]:
axes= [-2.5, 2.5, -1, 1.5]
for i, svm_clf in enumerate(svm_clfs):
    plt.subplot(221 + i)
    #plot_predictions(svm_clf, [-1.5, 2.5, -1, 1.5])
    plot_dataset(X, y,axes)
    gamma, C = hyperparams[i]
    plt.title(r"$\gamma = {}, C = {}$".format(gamma, C), fontsize=8)

# save_fig("moons_rbf_svc_plot")
y_decision=rbf_kernel_clf.decision_function(X)
x0s = np.linspace(axes[0], axes[1]).reshape(len(y_decision),1)
x1s = np.linspace(axes[2], axes[3]).reshape(len(y_decision),1)
x0, x1 = np.meshgrid(x0s, x1s)


plt.show()

In [ ]:
y_decision=rbf_kernel_clf.decision_function(X).reshape(8089,1)
plt.contourf(x0, x1, y_decision, cmap=plt.cm.brg, alpha=0.1)
plt.show()